In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
from tqdm import tqdm
from pylab import rcParams
import copy
import time
%matplotlib inline

In [ ]:
!git clone https://github.com/ultralytics/yolov3

In [ ]:
!hwinfo --short

In [ ]:
!python -m torch.utils.collect_env

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd /content/yolov3

In [ ]:
!sudo apt-get install python3.7

In [ ]:
!pip install -U -r requirements.txt

In [ ]:
!git clone https://github.com/NVIDIA/apex

In [ ]:
cd /content/yolov3/apex

In [ ]:
!pip install -v --no-cache-dir ./

In [ ]:
cd /content/

In [ ]:
!mkdir visual_genome

In [ ]:
cd visual_genome

In [ ]:
!mkdir images

In [ ]:
!mkdir labels

In [ ]:
cd /content/images

In [ ]:
!mkdir train

In [ ]:
cd /content/labels

In [ ]:
!mkdir train

In [ ]:
!mkdir test

In [ ]:
!mkdir val

In [ ]:
cd /content/

Downloading our data

In [ ]:
!wget https://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip

In [ ]:
!wget https://cs.stanford.edu/people/rak248/VG_100K_2/images.zip

In [ ]:
!wget http://visualgenome.org/static/data/dataset/objects_v1_2.json.zip

Unzipping our data

In [ ]:
!unzip "/content/objects_v1_2.json.zip" -d "/content"

In [ ]:
!unzip "/content/images.zip" -d "/content/visual_genome/images"

In [ ]:
!unzip "/content/images2.zip" -d "/content/visual_genome/images"

In [ ]:
!mv VG_100K_2/* train

In [ ]:
!mv VG_100K/* train

generation our training/validation/testing files 

In [ ]:
import json

In [ ]:
f = open("/content/objects.json","r") 

In [ ]:
data = json.load(f) 

In [ ]:
f.close()

In [ ]:
f1 = open("/content/yolov3/data/visualgenome.names","r") 
content = f1.readlines()
f1.close()

In [ ]:
f_image = open("/content/image_data.json","r") 
image_data = json.load(f_image) 
f_image.close()

In [ ]:
fim = open("/content/yolov3/data/visualgenome_train.txt","w")
fim.close()

In [ ]:
fim = open("/content/yolov3/data/visualgenome_test.txt","w")
fim.close()

In [ ]:
fim = open("/content/yolov3/data/visualgenome_val.txt","w")
fim.close()

training 60k images

In [ ]:
  import os
  image_ids=[]
  import os.path
  from os import path
  line = ""
  i = 0
  for sample in data:
    path_to=""
    
    for img in image_data: 
      if sample["image_id"]==img["image_id"]:
        image_width= img["width"]
        image_height= img["height"]

    if path.isfile("/content/visual_genome/images/train/"+str(sample["image_id"])+".jpg"):
      i+=1   
      path_to+= '/content/visual_genome/labels/train/'
      path_to+= str(sample["image_id"])
      path_to+='.txt'
      jsp='/content/visual_genome/labels/train'
      for obj in sample["objects"]:
          dw = 1./image_width
          dh = 1./image_height
          
          x_center = "{:.5f}".format((obj["x"]+(obj["w"]/2.0))*dw)
            
          y_center= "{:.5f}".format((obj["y"]+(obj["h"])/2.0)*dh)
          
          height = "{:.5f}".format(obj["h"]*dh)

          width = "{:.5f}".format(obj["w"]*dw)
          for name in obj["names"] :
            #  print(name + "|"+"x="+str(obj["x"])+"|y="+str(obj["y"])+"|w="+str(obj["w"])+"|h="+str(obj["h"])+"|"+str(x_min)+","+str(y_min)+","+str(x_max)+","+str(y_max))
            new_name=name+"\n"
            if new_name in content :
              if sample["image_id"] not in image_ids:
                image_ids.append(sample["image_id"])
                f_sample = open(path_to,"w") 
                f_sample.close()
                fim = open("/content/yolov3/data/visualgenome_train.txt","a")
                if os.stat("/content/yolov3/data/visualgenome_train.txt").st_size != 0 :
                  fim.write("\n")
                fim.write("/content/visual_genome/images/train/"+str(sample["image_id"])+".jpg")
                fim.close()  
              # ch = "x= {} y= {} image_id= {} name= {} x_norm= {} y_norm= {} ".format(obj["x"],obj["y"],sample["image_id"],name,x_center,y_center)
              #  print(ch)
              line=""
              obj_id = content.index(new_name)
              line+=str(obj_id)+" "+str(x_center)+" "+str(y_center)+" "+str(width)+" "+str(height)
              f_sample = open(path_to,"a") 
              if os.stat(path_to).st_size != 0 :
                  f_sample.write("\n")
              f_sample.write(line)
              f_sample.close()   
    print(i)
    if i==60000:
      break
  


testing 10k images

In [ ]:
  line = ""
  i = 0
  for sample in data:
    path_to=""
    
    for img in image_data: 
      if sample["image_id"]==img["image_id"]:
        image_width= img["width"]
        image_height= img["height"]

    if path.isfile("/content/visual_genome/images/train/"+str(sample["image_id"])+".jpg"):
      i+=1   
      path_to+= '/content/visual_genome/labels/test/'
      path_to+= str(sample["image_id"])
      path_to+='.txt'
      jsp='/content/visual_genome/labels/test'
      for obj in sample["objects"]:
          dw = 1./image_width
          dh = 1./image_height
          
          x_center = "{:.5f}".format((obj["x"]+(obj["w"]/2.0))*dw)
            
          y_center= "{:.5f}".format((obj["y"]+(obj["h"])/2.0)*dh)
          
          height = "{:.5f}".format(obj["h"]*dh)

          width = "{:.5f}".format(obj["w"]*dw)
          for name in obj["names"] :
            #  print(name + "|"+"x="+str(obj["x"])+"|y="+str(obj["y"])+"|w="+str(obj["w"])+"|h="+str(obj["h"])+"|"+str(x_min)+","+str(y_min)+","+str(x_max)+","+str(y_max))
            new_name=name+"\n"
            if new_name in content :
              if sample["image_id"] not in image_ids:
                image_ids.append(sample["image_id"])
                f_sample = open(path_to,"w") 
                f_sample.close()
                fim = open("/content/yolov3/data/visualgenome_test.txt","a")
                if os.stat("/content/yolov3/data/visualgenome_test.txt").st_size != 0 :
                  fim.write("\n")
                fim.write("/content/visual_genome/images/train/"+str(sample["image_id"])+".jpg")
                fim.close()  
              # ch = "x= {} y= {} image_id= {} name= {} x_norm= {} y_norm= {} ".format(obj["x"],obj["y"],sample["image_id"],name,x_center,y_center)
              #  print(ch)
              line=""
              obj_id = content.index(new_name)
              line+=str(obj_id)+" "+str(x_center)+" "+str(y_center)+" "+str(width)+" "+str(height)
              f_sample = open(path_to,"a") 
              if os.stat(path_to).st_size != 0 :
                  f_sample.write("\n")
              f_sample.write(line)
              f_sample.close()   
    print(i)
    if i==10000:
      break

validation 30k images

In [ ]:
  line = ""
  i = 0
  for sample in data:
    path_to=""
    
    for img in image_data: 
      if sample["image_id"]==img["image_id"]:
        image_width= img["width"]
        image_height= img["height"]

    if path.isfile("/content/visual_genome/images/train/"+str(sample["image_id"])+".jpg"):
      i+=1   
      path_to+= '/content/visual_genome/labels/val/'
      path_to+= str(sample["image_id"])
      path_to+='.txt'
      jsp='/content/visual_genome/labels/val'
      for obj in sample["objects"]:
          dw = 1./image_width
          dh = 1./image_height
          
          x_center = "{:.5f}".format((obj["x"]+(obj["w"]/2.0))*dw)
            
          y_center= "{:.5f}".format((obj["y"]+(obj["h"])/2.0)*dh)
          
          height = "{:.5f}".format(obj["h"]*dh)

          width = "{:.5f}".format(obj["w"]*dw)
          for name in obj["names"] :
            #  print(name + "|"+"x="+str(obj["x"])+"|y="+str(obj["y"])+"|w="+str(obj["w"])+"|h="+str(obj["h"])+"|"+str(x_min)+","+str(y_min)+","+str(x_max)+","+str(y_max))
            new_name=name+"\n"
            if new_name in content :
              if sample["image_id"] not in image_ids:
                image_ids.append(sample["image_id"])
                f_sample = open(path_to,"w") 
                f_sample.close()
                fim = open("/content/yolov3/data/visualgenome_val.txt","a")
                if os.stat("/content/yolov3/data/visualgenome_val.txt").st_size != 0 :
                  fim.write("\n")
                fim.write("/content/visual_genome/images/train/"+str(sample["image_id"])+".jpg")
                fim.close()  
              # ch = "x= {} y= {} image_id= {} name= {} x_norm= {} y_norm= {} ".format(obj["x"],obj["y"],sample["image_id"],name,x_center,y_center)
              #  print(ch)
              line=""
              obj_id = content.index(new_name)
              line+=str(obj_id)+" "+str(x_center)+" "+str(y_center)+" "+str(width)+" "+str(height)
              f_sample = open(path_to,"a") 
              if os.stat(path_to).st_size != 0 :
                  f_sample.write("\n")
              f_sample.write(line)
              f_sample.close()   
    print(i)
    if i==30000:
      break

In [ ]:
cd /content/

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.set_device(0)

In [ ]:
cd /content/yolov3

training and testing for 175 epochs

In [ ]:
!python3 train.py --cfg yolov3.cfg --batch-size 32  --epochs 175 --data /content/yolov3/data/visualgenome.data --weights yolov3.pt

In [ ]:
cd /content/yolov3/

Model pruning added to train.py

In [ ]:
def weight_prune(model, pruning_percentage, test_dl):
    model1 = copy.deepcopy(model)
    length = len(list(model1.parameters()))
    for i, param in enumerate(model1.parameters()):
        if len(param.size())!=1 and i<length-2:
            weight = param.detach().cpu().numpy()
            weight[np.abs(weight)<np.percentile(np.abs(weight), pruning_percentage)] = 0
            weight = torch.from_numpy(weight).to(device)
            param.data = weight
    return test(model1, test_dl)

In [ ]:
pruning_percent = [0, 25, 50, 60, 70, 80, 90, 95, 97, 99]


In [ ]:
accuracy_weight_pruning = []
for percent in pruning_percent:
    accuracy_weight_pruning.append(weight_prune(model, percent, test_dl)

In [ ]:
rcParams['figure.figsize'] = 12, 8
plt.plot(pruning_percent, np.size(pruning_percent)*[original_accuracy], 'r',
         pruning_percent, accuracy_weight_pruning, 'b')
plt.grid()
plt.legend([['Original Accuracy'], 
            ['Accuracy with weight pruning']],
           loc='lower left', fontsize='xx-large')
plt.xlabel('Pruning Percentage', fontsize='xx-large')
plt.ylabel('Accuracy', fontsize='xx-large')
plt.xticks(pruning_percent)
plt.yticks(np.arange(0, 1.05, 0.05))
plt.show()

In [ ]:
def neuron_pruning(model, pruning_percentage, test_dl):
    model1 = copy.deepcopy(model)
    length = len(list(model1.parameters()))
    for i, param in enumerate(model1.parameters()):
        if len(param.size())!=1 and i<length-2:
            weight = param.detach().cpu().numpy()
            norm = np.linalg.norm(weight, axis=0)
            weight[:, np.argwhere(norm<np.percentile(norm, pruning_percentage))] = 0
            weight = torch.from_numpy(weight).to(device)
            param.data = weight
    return test(model1, test_dl)

In [ ]:
accuracy_neuron_pruning = []
for percent in pruning_percent:
    accuracy_neuron_pruning.append(neuron_pruning(model, percent, test_dl))

detection

In [ ]:
from utils import utils; utils.plot_results()

In [ ]:
!python3 detect.py --cfg yolov3.cfg  --source /content/yolov3/data/samples/285604.jpg

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img=mpimg.imread('/content/yolov3/output/285604.jpg')
imgplot = plt.imshow(img)
plt.show()